# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

The dataset is from the Kaggle dataset ["HR Analytics: Job Change of Data Scientists"](https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists). It lists individuals who have gone through the company's Data Science training program, some demographic and background information on the trainee, and wether or not that employee churned from the company. I will be building an ML pipeline to predict the likelihood of a new candidate churning, with an API accessible to run on an as-needed basis.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'cp-experiment'

experiment=Experiment(ws, experiment_name)

In [3]:
compute_target = ComputeTarget(workspace=ws, name='cap-compute')

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

This is a classification task, since the response variable is a yes/no retention indicator. The experiment timeout is set to 20 minutes, as that seems to be a reasonable time to operate on. 5 concurrent iterations because the compute cluster is at 6 nodes max. The blocked models is because of the probmen specified here: https://knowledge.udacity.com/questions/509841

In [4]:
# TODO: Put your automl settings here
automl_settings = dict(
    compute_target=compute_target,
    task="classification",
    training_data=ws.datasets['DsEmpData'],
    label_column_name="target",   
    path='./capstone-project',
    enable_early_stopping=True,
    featurization='auto',
    debug_log="automl_errors.log",
    experiment_timeout_minutes=20,
    max_concurrent_iterations=5,
    primary_metric='accuracy',
    blocked_models=['XGBoostClassifier']
)

# TODO: Put your automl config here
automl_config = AutoMLConfig(**automl_settings)

In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

AutoMLException: AutoMLException:
	Message: Could not find a model with valid score for metric 'accuracy'. Please ensure that at least one run was successfully completed with a valid score for the given metric.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Could not find a model with valid score for metric 'accuracy'. Please ensure that at least one run was successfully completed with a valid score for the given metric.",
        "target": "metric",
        "inner_error": {
            "code": "NotFound",
            "inner_error": {
                "code": "ModelMissing"
            }
        }
    }
}

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
published_pipeline = hyperdrive_run.publish_pipeline(
    name='DS Training Retention',
    description="DS Training Model Endpoint",
    version='1.0.0',
)
published_pipeline

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service